In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
import pandas as pd
from IPython.display import display, Image
import shutil

import os
import re
import json
from pylabel import importer
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
pd.set_option('display.max_columns', None)


In [ ]:
HOME = os.getcwd()
train_path = "../../data/material_version/train"
valid_path = "../../data/material_version/val"

train_annots = "../../data/material_version/instances_train_trashcan.json"
valid_annots = "../../data/material_version/instances_val_trashcan.json"

destination_path = f"{HOME}/destination"

results = "reports/results"

In [ ]:
class ScriptUtilities:
    @staticmethod
    def json_viewer(annots_json):
        # Read the JSON file
        with open(annots_json, 'r') as file:
            annotations_data = json.load(file)

        # Display information for the top 5 images
        for image_info in annotations_data['images'][:5]:
            print(f"Image ID: {image_info['id']}")
            print(f"File Name: {image_info['file_name']}")
            print(f"Height: {image_info['height']}, Width: {image_info['width']}")
            print(f"Date Captured: {image_info['date_captured']}")
            print("----------------------")

        # View content under 'annotations' key
        if 'annotations' in annotations_data:
            print("Annotations:")
            for annotation in annotations_data['annotations'][:5]:
                print(annotation)
                print("----------------------")
        else:
            print("No 'annotations' key found in the JSON data.")

    @staticmethod
    def json_tree(annots_json):
        # Read the JSON file
        with open(annots_json, 'r') as file:
            annotations_data = json.load(file)
        # Print the keys at the top level of the JSON structure
        print(annotations_data.keys())

    @staticmethod
    def coco_to_yolo_converter(annots_json, path_to_images, name):
        dataset = importer.ImportCoco(annots_json, path_to_images=path_to_images, name=name)

        return dataset

In [ ]:
# Train dataset
train_dataset = ScriptUtilities.coco_to_yolo_converter(train_annots, train_path, name="BCCD_train_coco")

# Valid dataset
valid_dataset = ScriptUtilities.coco_to_yolo_converter(valid_annots, valid_path, name="BCCD_valid_coco")

In [ ]:
train_dataset.df.sample(5)


In [ ]:
valid_dataset.df.sample(5)


In [ ]:
dataset_list = [train_dataset, valid_dataset]

for dataset in dataset_list:
    print("\n")
    print(f"Number of images: {dataset.analyze.num_images}")
    print(f"Number of classes: {dataset.analyze.num_classes}")
    print(f"Classes:{dataset.analyze.classes}")
    print(f"Class counts:\n{dataset.analyze.class_counts}")
    print(f"Path to annotations:\n{dataset.path_to_annotations}")

In [ ]:
display(train_dataset.visualize.ShowBoundingBoxes(2))

In [ ]:
train_dataset.export.ExportToYoloV5(output_path = f'{destination_path}/training/labels',
                              copy_images = True,
                              yaml_file = "dataset.yaml",
                              segmentation = True)

In [ ]:
valid_dataset.export.ExportToYoloV5(output_path = f'{destination_path}/valid/labels',
                              copy_images = True,
                              yaml_file = "dataset.yaml",
                              segmentation = True)

In [ ]:
# Note!!! Only run this code once
path = f"destination/training/labels" #path of labels
labels = os.listdir(path)
for x in labels:
    lines = list()
    with open(path+"/"+x, "r+") as f:
        for line in f.read().splitlines():
            split_line = line.split(" ")  # split on space character (and remove newline characters as well)
            split_line[0] = str(
              int(split_line[0]) - 1)  # update the value inside the loop. the loop used in later not needed.
            lines.append(split_line)  # add split list into list of lines

    with open(path+"/"+x, 'w') as file:  # rewrite to file
        for line in lines:
            write_me = ' '.join(line)  # Use join method to add the element together
            file.write(write_me + "\n")

# Code adapted from: hhttps://www.kaggle.com/code/vencerlanz09/insect-images-classification-using-yolov5

In [ ]:
# Note!!! Only run this code once
path = f"destination/valid/labels" #path of labels
labels = os.listdir(path)
for x in labels:
    lines = list()
    with open(path+"/"+x, "r+") as f:
        for line in f.read().splitlines():
            split_line = line.split(" ")  # split on space character (and remove newline characters as well)
            split_line[0] = str(
              int(split_line[0]) - 1)  # update the value inside the loop. the loop used in later not needed.
            lines.append(split_line)  # add split list into list of lines

    with open(path+"/"+x, 'w') as file:  # rewrite to file
        for line in lines:
            write_me = ' '.join(line)  # Use join method to add the element together
            file.write(write_me + "\n")

# Code adapted from: hhttps://www.kaggle.com/code/vencerlanz09/insect-images-classification-using-yolov5

In [ ]:
os.environ["WANDB_MODE"]="offline"

In [ ]:
!rm -f ../destination/training/labels.cache

In [ ]:
import requests

url = "https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n-seg.pt"
output_path = "yolov8n-seg.pt"

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {output_path}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


In [ ]:
# Load a model
model = YOLO(f"yolov8n-seg.pt")  # load a pretrained model (recommended for training)

In [ ]:
# Train the model
results = model.train(data=f"destination/training/dataset.yaml", epochs=35, imgsz=(480, 270))

In [ ]:
# Path to the PNG file
result_png_path = "../models/runs/segment/train15/results.png"

# Display the PNG file
Image(result_png_path)
